## K-Prototypes Part II - Applied

In the first section of this exploration of clustereing with mixed data, I covered some of the theroetical reasons for _why_ straight up K-Means is not aprpopriate for mixed data. I also wrote about some of the racticaly reasons for wanting to cluster data in a business conetxt.
<br> For this subsequent continuation of the tutorial, I will attempt to apply the implementation of  [K-Protoypes]( https://github.com/nicodv/kmodes#huang97) to a data set I have not worked with before. My only _a priori_ requierement is that it had to have mixed continuous and categorical data 

## KDD Cup 1999 Data 

Scikit learned has several pre-canned data sets you can play around with. I chose the KDD Cup 1999 Data set. According to the publishers of this data:

<br>_"This is the data set used for The Third International Knowledge Discovery and Data Mining Tools Competition, which was held in conjunction with KDD-99 The Fifth International Conference on Knowledge Discovery and Data Mining. The competition task was to build a network intrusion detector, a predictive model capable of distinguishing between 'bad' connections, called intrusions or attacks, and 'good' normal connections. This database contains a standard set of data to be audited, which includes a wide variety of intrusions simulated in a military network environment."_ <br>
<br>
A few notes on the options I applied to loaing this data set. This is simulated data and contains many more 'rare' events than real world intrusion data would. One of the goals of clustering I mentioned in Part I is the fact that it can be used to detect groups that over-index on outliers and rare events that you may eventually want to model.However, there is an option (the 'SA' subset for resampling the data that drastically cuts down the proportion of 'bad' connections and allows


In [3]:
import sklearn.datasets
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#Load up a small sample of the 'SA' subset of KDD Cup 1999
kddcup=sklearn.datasets.fetch_kddcup99 (subset='SA', random_state=666, percent10=True)

In [4]:
kddcup

{'data': array([[0, b'tcp', b'http', ..., 0.0, 0.0, 0.0],
        [0, b'tcp', b'http', ..., 0.0, 0.0, 0.0],
        [0, b'tcp', b'http', ..., 0.0, 0.0, 0.0],
        ...,
        [0, b'icmp', b'ecr_i', ..., 0.0, 0.0, 0.0],
        [0, b'icmp', b'ecr_i', ..., 0.0, 0.0, 0.0],
        [0, b'icmp', b'ecr_i', ..., 0.0, 0.0, 0.0]], dtype=object),
 'target': array([b'normal.', b'normal.', b'normal.', ..., b'smurf.', b'smurf.',
        b'smurf.'], dtype=object)}

In [5]:
kddcup['data']

array([[0, b'tcp', b'http', ..., 0.0, 0.0, 0.0],
       [0, b'tcp', b'http', ..., 0.0, 0.0, 0.0],
       [0, b'tcp', b'http', ..., 0.0, 0.0, 0.0],
       ...,
       [0, b'icmp', b'ecr_i', ..., 0.0, 0.0, 0.0],
       [0, b'icmp', b'ecr_i', ..., 0.0, 0.0, 0.0],
       [0, b'icmp', b'ecr_i', ..., 0.0, 0.0, 0.0]], dtype=object)

In [7]:

import numpy as np
# get some of the specifics of the dataset
X = kddcup.data
y = kddcup.target != b'normal.'


In [9]:
y

array([False, False, False, ...,  True,  True,  True])